In [ ]:
!git clone https://github.com/google-research/timesfm.git
!cd timesfm && pip install .[torch]

In [ ]:
! pip install utilsforecast

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
import timesfm

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/refs/heads/main/data/Walmart.csv"

df = pd.read_csv(DATA_URL)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

df.head()

## Zero-shot forecasting

In [ ]:
HORIZON = 13

In [ ]:
# Initialize TimesFM-2.5

# Compile model


In [ ]:
# Get all sorted unique ids

# Prepare list of inputs


for store_id in unique_stores:
    # Append input data for each unique id

# Get point forecasts and quantile forecasts


In [ ]:
quantile_forecast.shape

In [ ]:
# Create a DataFrame from TimesFM output
def create_forecast_df(
    quantile_forecast_array: np.array,
    original_df: pd.DataFrame,
    id_col: str,
    time_col: str,
    target_col: str,
    horizon: int,
    freq: str,
):

    # Extract num_series, horizon, and number of quantiles

    # List for all forecast rows


    unique_ids = sorted(original_df[id_col].unique())


    for i, id in enumerate(unique_ids):
        # Get the last known date for this store from the original DataFrame

        # Generate forecast dates

        for h in range(horizon):
            # Create forecast row

            # Append all rows

    return pd.DataFrame(all_forecast_rows)

In [ ]:
# Create DataFrame from TimesFM output


In [ ]:
plot_series(
    df=df,
    forecasts_df=fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

## Cross-validation

In [ ]:
# Cross-validation with TimesFM
def timesfm_cv(
    df: pd.DataFrame,
    model: timesfm.timesfm_2p5.timesfm_2p5_torch.TimesFM_2p5_200M_torch,
    horizon: int,
    n_windows: int,
    id_col: str,
    time_col: str,
    target_col: str,
    freq: str,
):
    # List for all cross-validation results

    # Get latest date

    for i in range(n_windows):
        # Calculate the cutoff date for the current window

        # Create a training DataFrame up to the cutoff_date

        # Prepare inputs_list


        # Generate quantile forecasts


        # Convert forecasts to DataFrame
        fcsts_df = create_forecast_df(
            quantile_forecast_array=quantile_forecast,
            original_df=df_train,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            horizon=horizon,
            freq=freq,
        )

        # Add cutoff column

        # Append results

    # Concatenate all DataFrames

    # Merge actual values

    return cv_df

In [ ]:
# Run cross-validation

cv_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_df.drop(columns=["cutoff", "Weekly_Sales"]),
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
eval_df = evaluate(
    cv_df.drop(columns=["cutoff"]),
    metrics=[mae, smape],
    models=['timesfm'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df

## Forecasting with covariates

In [ ]:
!pip install "jax[cuda]" scikit-learn

In [ ]:
model = timesfm.TimesFM_2p5_200M_torch.from_pretrained("google/timesfm-2.5-200m-pytorch")

model.compile(
    timesfm.ForecastConfig(
        max_context=1024,
        max_horizon=256,
        normalize_inputs=True,
        use_continuous_quantile_head=True,
        force_flip_invariance=True,
        infer_is_positive=True,
        fix_quantile_crossing=True,
        # Set return_backcast to use exogenous features

    )
)

In [ ]:
# Add exogenous feature to cross-validation
def timesfm_cv(
    df: pd.DataFrame,
    model: timesfm.timesfm_2p5.timesfm_2p5_torch.TimesFM_2p5_200M_torch,
    horizon: int,
    n_windows: int,
    id_col: str,
    time_col: str,
    target_col: str,
    freq: str,
    # Specify list of columns of known future covariates

):
    all_cv_forecasts = []

    max_date = df[time_col].max()

    for i in range(n_windows):
        # Calculate the cutoff date for the current window
        cutoff_date = max_date - pd.Timedelta((n_windows - i) * horizon, unit=freq[0])

        # Create a training DataFrame up to the cutoff_date
        df_train = df[df[time_col] <= cutoff_date]

        # Prepare inputs_list
        unique_ids = sorted(df_train[id_col].unique())
        inputs_list = []
        for id in unique_ids:
            sub_df = df_train[df_train[id_col] == id].sort_values(by=time_col)
            inputs_list.append(sub_df[target_col].values)

        # Prepare dynamic categorical covariates if specified


            for col_name in dynamic_categorical_cols:
                covariate_values = []

                for id in unique_ids:
                    # Get the full time series for this ID (train + horizon)

                    # Filter to get: historical data up to cutoff + horizon periods after cutoff


                    # Extract covariate values (input_size + horizon)


                # Assign in dictionary

        # Generate forecasts
        if dynamic_categorical_covariates:


            # Stack the list of arrays into a single array

        # The rest of the function stays the same
        else:
            _, quantile_forecast = model.forecast(
                horizon=horizon,
                inputs=inputs_list
            )

        # Convert forecasts to DataFrame
        fcsts_df = create_forecast_df(
            quantile_forecast_array=quantile_forecast,
            original_df=df_train,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            horizon=horizon,
            freq=freq,
        )

        # Add cutoff column
        fcsts_df['cutoff'] = cutoff_date

        all_cv_forecasts.append(fcsts_df)

    cv_df = pd.concat(all_cv_forecasts, ignore_index=True)
    cv_df = cv_df.merge(df[[id_col, time_col, target_col]], how="left", on=[id_col, time_col])
    return cv_df

In [ ]:
# Cross-validation with covariates

cv_df_exog.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_df_exog.drop(columns=["cutoff", "Weekly_Sales"]),
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
exog_eval_df = evaluate(
    cv_df_exog.drop(columns=["cutoff"]),
    metrics=[mae, smape],
    models=['timesfm'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
exog_eval_df